In [2]:
%matplotlib inline

import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np
import matplotlib.pyplot as plt
import re
import operator

'''
/home/dhankar/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'''

'\n/home/dhankar/anaconda2/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.\n  "This module will be removed in 0.20.", DeprecationWarning)\n\n\n'

In [56]:
#
dfTrn = pd.read_csv("train.csv") # On Local Machine 
dfTest = pd.read_csv("test.csv") # On Local Machine 
#dfTrn = pd.read_csv("../input/train.csv") # On Kaggle 
#dfTest = pd.read_csv("../input/test.csv") # On Kaggle 
print('Number of rows: {}, Number of columns: {}'.format(*dfTrn.shape))
print ("_"*90)
print('Number of rows: {}, Number of columns: {}'.format(*dfTest.shape))
print ("_"*90)
print(dfTrn.head(5))
print ("_"*90)
print(dfTest.head(5))

Number of rows: 891, Number of columns: 12
__________________________________________________________________________________________
Number of rows: 418, Number of columns: 11
__________________________________________________________________________________________
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  

In [57]:
feature_labels = []
missing_values = []

for col in dfTrn.columns:
    feature_labels.append(col)
    missing_values.append(dfTrn[col].isnull().values.ravel().sum()) # Append Feature wise Missing Values  
    print(col,"=", missing_values[-1]) # prints Feature Labels with Missing Values Count ...

#Source - http://stackoverflow.com/questions/28199524/best-way-to-count-the-number-of-rows-with-missing-values-in-a-pandas-dataframe    

PassengerId = 0
Survived = 0
Pclass = 0
Name = 0
Sex = 0
Age = 177
SibSp = 0
Parch = 0
Ticket = 0
Fare = 0
Cabin = 687
Embarked = 2


In [58]:
#Source - Kaggle - https://www.kaggle.com/michielkalkman/titanic/kaggle-titanic-001

# df_Any is any data_set TRAIN or TEST passed to this function

def harmonize_data(df_Any):
    
    df_Any["Age"] = df_Any["Age"].fillna(df_Any["Age"].median()) # Impute missing values
#    
    df_Any.loc[df_Any["Sex"] == "male", "Sex"] = 0
    df_Any.loc[df_Any["Sex"] == "female", "Sex"] = 1
#    
    df_Any["Embarked"] = df_Any["Embarked"].fillna("S") # Impute missing values
#
    df_Any.loc[df_Any["Embarked"] == "S", "Embarked"] = 0
    df_Any.loc[df_Any["Embarked"] == "C", "Embarked"] = 1
    df_Any.loc[df_Any["Embarked"] == "Q", "Embarked"] = 2

    df_Any["Fare"] = df_Any["Fare"].fillna(df_Any["Fare"].median())

    return df_Any

In [59]:
# Get Harmonized DF's from Train and Test Data 
# Cant be running this code cell or function more than once for a particular - df_Any # Test or Train 

df_harmonized_train = harmonize_data(dfTrn)
print(df_harmonized_train.head(10))

print ("_"*90)
df_harmonized_test = harmonize_data(dfTest)
print(df_harmonized_test.head(10))

names = df_harmonized_test.columns.values
print ("_"*90)
print (names)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris   0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   1  38.0      1      0   
2                             Heikkinen, Miss. Laina   1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)   1  35.0      1      0   
4                           Allen, Mr. William Henry   0  35.0      0      0   
5                                   Moran, Mr. James   0  28.0      0      0   
6                            McCarthy, Mr. 

In [ ]:
#Source - Kaggle - https://www.kaggle.com/michielkalkman/titanic/kaggle-titanic-001

def create_submission(alg, train, test, predictors, filename):

    alg.fit(train[predictors], train["Survived"]) # Fit whatever Algorithm passed to "alg"
    predictions = alg.predict(test[predictors])   # Make predictions on TEST data - dfTest - save "predictions" 

    submission = pd.DataFrame({                   # Create "submission" DF ,Two Features- Pid and Preds for SURVIVAL
        "PassengerId": test["PassengerId"],
        "Survived": predictions
    })
    
    submission.to_csv(filename, index=False)      # "Filename" parameter passed by user - save CSV 

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

alg    = LogisticRegression(random_state=1)
scores = cross_validation.cross_val_score(
    alg,
    df_harmonized_train[predictors],
    df_harmonized_train["Survived"],
    cv=3
)

print(scores.mean())

# Check how "predictors" can be "names" from above - not to to be typed manually 
#


0.787878787879


In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

alg = RandomForestClassifier(       # Various parameter values passed :- 
    random_state=124,               # 1 , 123 , 124
    n_estimators=250,               # 150 , 250 , 
    min_samples_split=6,            # 4 , 6  ##################################
    min_samples_leaf=2              # 2 , 2
)

                                    #> 0.820426487093 #> 0.832833437419 #> 0.83397558283 #> 0.835056689342

scores = cross_validation.cross_val_score(
    alg,
    df_harmonized_train[predictors],
    df_harmonized_train["Survived"],
    cv=18                            # 3, 5 , 15, 18 Nothing else changed == 0.835056689342
)

print(scores.mean())



0.835056689342


In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation

predictors = ["Pclass", "Sex", "Age", "Fare"]

alg = RandomForestClassifier(       # Various parameter values passed :- 
    random_state=124,               # 124
    n_estimators=250,               # 250 
    min_samples_split=6,            # 6  
    min_samples_leaf=2              # 2
)

                                    #> 0.842857142857

scores = cross_validation.cross_val_score(
    alg,
    df_harmonized_train[predictors],
    df_harmonized_train["Survived"],
    cv=18                            # 18 Nothing else changed == 
)

print(scores.mean())



0.842857142857


In [83]:
predictors = ["Pclass", "Sex", "Age", "Fare"]

alg = RandomForestClassifier(       # Various parameter values passed :- 
    random_state=124,               # 124
    n_estimators=250,               # 250 
    min_samples_split=6,            # 6  
    min_samples_leaf=2              # 2
)

'''
algorithms = [
    [GradientBoostingClassifier(random_state=123, n_estimators=25, max_depth=3), predictors],
    [LogisticRegression(random_state=123), ["Pclass", "Sex", "Age", "Fare"]]
]

full_predictions = []
for alg, predictors in algorithms:
    alg.fit(df_harmonized_train[predictors], df_harmonized_train["Survived"])
    predictions = alg.predict_proba(df_harmonized_test[predictors].astype(float))[:,1]
    full_predictions.append(predictions)

predictions = (full_predictions[0] * 3 + full_predictions[1]) / 4

predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0
predictions = predictions.astype(int)

submission = pd.DataFrame({
        "PassengerId": df_harmonized_test["PassengerId"],
                               "Survived": predictions
    })
submission.to_csv('Submission_Titanic.csv',index=True)
'''

#full_predictions = []

alg.fit(df_harmonized_train[predictors], df_harmonized_train["Survived"])
predictions = alg.predict_proba(df_harmonized_test[predictors].astype(float))[:,1]
#full_predictions.append(predictions)

#predictions = (full_predictions[0] * 3 + full_predictions[1]) / 4

predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0
predictions = predictions.astype(int)

submission = pd.DataFrame({
        "PassengerId": df_harmonized_test["PassengerId"],
                               "Survived": predictions
    })
submission.to_csv('Submission_Titanic2.csv',index=True)

In [ ]:


submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)



In [76]:
# Make predictions using the test set.
predictions = alg.predict(df_harmonized_test[predictors])


NotFittedError: Estimator not fitted, call `fit` before exploiting the model.

In [74]:
submission = pd.DataFrame({
        "PassengerId": df_harmonized_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv('Titanic_1.csv', index=False)

NameError: name 'predictions' is not defined

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation

predictors = ["Pclass", "Sex", "Title", "Fare"]

alg = RandomForestClassifier(       # Various parameter values passed :- 
    random_state=124,               # 124
    n_estimators=250,               # 250 
    min_samples_split=6,            # 6  
    min_samples_leaf=2              # 2
)

                                    #> 0.842857142857

scores = cross_validation.cross_val_score(
    alg,
    df_harmonized_train[predictors],
    df_harmonized_train["Survived"],
    cv=18                            # 18 Nothing else changed == 
)

print(scores.mean())



KeyError: "['Title'] not in index"

In [9]:
dft = pd.read_csv('train.csv', index_col='PassengerId')
print('Number of rows: {}, Number of columns: {}'.format(*dft.shape))
print ("_"*90)
print(dft.head(5))


Number of rows: 891, Number of columns: 11
__________________________________________________________________________________________
             Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin Embarked  
Pas

In [10]:
X = pd.DataFrame()
y = train['Survived']

In [11]:
X

""


In [12]:
y

PassengerId
1      0
2      1
3      1
4      1
5      0
6      0
7      0
8      0
9      1
10     1
11     1
12     1
13     0
14     0
15     0
16     1
17     0
18     1
19     0
20     1
21     0
22     1
23     1
24     1
25     0
26     1
27     0
28     0
29     1
30     0
      ..
862    0
863    1
864    0
865    0
866    1
867    1
868    0
869    0
870    1
871    0
872    1
873    0
874    0
875    1
876    1
877    0
878    0
879    0
880    1
881    1
882    0
883    0
884    0
885    0
886    0
887    0
888    1
889    0
890    1
891    0
Name: Survived, dtype: int64